<a href="https://colab.research.google.com/github/AfzalKamboh/Music_Recommandation_System/blob/main/MusicRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity
import requests
import base64
import pandas as pd

In [ ]:

# Replace with your own Client ID and Client Secret
CLIENT_ID = 'e8ef0c5dc94b4c39a6dbff1609429ad1'
CLIENT_SECRET = 'fa1d3d6c61b4476394737257ef6a9a51'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [ ]:
playlist_id = '37i9dQZF1DXcBWIGoYBM5M'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
music_df

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Enough Is Enough,Post Malone,AUSTIN,6r1lh7fHMB499vGKtIyJLy,3BHFResGQiUvbYToUdaDQz,82,2023-07-28,165175,True,https://open.spotify.com/track/3BHFResGQiUvbYT...,...,0.7680,0,-4.911,1,0.0344,0.01400,0.000000,0.1090,0.3320,166.061
1,K-POP,"Travis Scott, Bad Bunny, The Weeknd",K-POP,10eNMeTimUzAKLhCOpIUlb,5L3ecxQnQ9qTBmnLQiwf0C,93,2023-07-21,185221,True,https://open.spotify.com/track/5L3ecxQnQ9qTBmn...,...,0.6180,5,-5.614,0,0.0619,0.03860,0.000003,0.6710,0.5350,121.075
2,Barbie World (with Aqua) [From Barbie The Album],"Nicki Minaj, Ice Spice, Aqua",Barbie World (with Aqua) [From Barbie The Album],5CM66hwjlbZ06LhONWXOAs,741UUVE2kuITl0c6zuqqbO,94,2023-06-23,109750,True,https://open.spotify.com/track/741UUVE2kuITl0c...,...,0.5800,0,-8.393,1,0.2470,0.51900,0.000127,0.2330,0.7530,144.072
3,Dance The Night (From Barbie The Album),Dua Lipa,Dance The Night (From Barbie The Album),5cH7FqB7JD5q1tJXJ7FHYu,1vYXt7VSjH9JIM5oRRo7vA,96,2023-05-25,176579,False,https://open.spotify.com/track/1vYXt7VSjH9JIM5...,...,0.8450,11,-4.930,0,0.0480,0.02070,0.000000,0.3290,0.7750,110.056
4,vampire,Olivia Rodrigo,vampire,5kqfR7EuGbyp8x27Pr1kY9,3k79jB4aGmMDUQzEwa46Rz,98,2023-06-30,219724,True,https://open.spotify.com/track/3k79jB4aGmMDUQz...,...,0.5320,5,-5.745,1,0.0560,0.16900,0.000000,0.3110,0.3220,137.827
5,What Was I Made For? [From The Motion Picture ...,Billie Eilish,What Was I Made For? [From The Motion Picture ...,3AafSrFIbJPH6BJHiJm1Cd,6wf7Yu7cxBSPrRlWeSeK0Q,96,2023-07-13,222370,False,https://open.spotify.com/track/6wf7Yu7cxBSPrRl...,...,0.0911,0,-17.665,1,0.0307,0.95900,0.000001,0.0980,0.1420,78.403
6,Popular (with Playboi Carti & Madonna) - Music...,"The Weeknd, Playboi Carti, Madonna",Popular (Music from the HBO Original Series),5O0zUvdnJr0RbWzLFneN2i,6WzRpISELf3YglGAh7TXcG,93,2023-06-02,215467,True,https://open.spotify.com/track/6WzRpISELf3YglG...,...,0.6780,1,-6.276,1,0.1890,0.05690,0.000037,0.4120,0.8520,99.029
7,Super Shy,NewJeans,NewJeans 'Super Shy',5V729UqvhwNOcMejx0m55I,5sdQOyqq2IDhvmx2lHOpwd,95,2023-07-07,154667,False,https://open.spotify.com/track/5sdQOyqq2IDhvmx...,...,0.8170,5,-6.018,0,0.0748,0.18000,0.000034,0.1460,0.5150,149.921
8,Seven (feat. Latto) (Explicit Ver.),"Jung Kook, Latto",Seven (feat. Latto),53985D8g3JcGBoULSOYYKX,7x9aauaA9cu6tyfpHnqDLo,100,2023-07-14,185427,True,https://open.spotify.com/track/7x9aauaA9cu6tyf...,...,0.8320,11,-4.107,1,0.0434,0.31100,0.000000,0.0815,0.8900,124.997
9,Cruel Summer,Taylor Swift,Lover,1NAmidJlEaVgA3MpcPFYGq,1BxfuPKGuaTgP7aM0Bbdwr,99,2019-08-23,178427,False,https://open.spotify.com/track/1BxfuPKGuaTgP7a...,...,0.7020,9,-5.707,1,0.1570,0.11700,0.000021,0.1050,0.5640,169.994


In [ ]:
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Track Name        50 non-null     object 
 1   Artists           50 non-null     object 
 2   Album Name        50 non-null     object 
 3   Album ID          50 non-null     object 
 4   Track ID          50 non-null     object 
 5   Popularity        50 non-null     int64  
 6   Release Date      50 non-null     object 
 7   Duration (ms)     50 non-null     int64  
 8   Explicit          50 non-null     bool   
 9   External URLs     50 non-null     object 
 10  Danceability      50 non-null     float64
 11  Energy            50 non-null     float64
 12  Key               50 non-null     int64  
 13  Loudness          50 non-null     float64
 14  Mode              50 non-null     int64  
 15  Speechiness       50 non-null     float64
 16  Acousticness      50 non-null     float64
 17 

In [ ]:
music_df.isnull().sum()

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64

In [ ]:
music_df.describe()

,Popularity,Duration (ms),Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,92.640000,192346.820000,0.659920,0.655202,4.920000,-6.497360,0.640000,0.071174,0.266926,0.025184,0.183456,0.510878,125.665600
std,3.538477,35178.349601,0.114595,0.167959,3.584775,2.287555,0.484873,0.062338,0.262632,0.096423,0.130997,0.243947,27.546033
min,82.000000,109750.000000,0.444000,0.091100,0.000000,-17.665000,0.000000,0.025400,0.001260,0.000000,0.032200,0.096900,66.900000
25%,90.250000,167870.500000,0.559500,0.558250,1.000000,-7.403500,0.000000,0.034600,0.057925,0.000000,0.099000,0.304500,102.642250
50%,92.000000,187027.000000,0.667500,0.676500,5.000000,-5.927000,1.000000,0.048200,0.159500,0.000005,0.115000,0.516500,127.992000
75%,95.000000,219656.250000,0.740000,0.789750,8.000000,-5.301250,1.000000,0.073050,0.453000,0.000162,0.249500,0.717750,140.853000
max,100.000000,274138.000000,0.916000,0.965000,11.000000,-3.404000,1.000000,0.327000,0.959000,0.629000,0.671000,0.964000,203.853000


In [ ]:
data = music_df

In [ ]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

In [ ]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key',
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [ ]:
# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [ ]:
# a function to get hybrid recommendations based on weighted popularity
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec
    hybrid_recommendations = hybrid_recommendations.append({
        'Track Name': input_song_name,
        'Artists': music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0],
        'Album Name': music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0],
        'Release Date': music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0],
        'Popularity': weighted_popularity_score
    }, ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]


    return hybrid_recommendations

In [ ]:
input_song_name = "Only Girl (In the World)"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
recommendations

'Only Girl (In the World)' not found in the dataset. Please enter a valid song name.
Hybrid recommended songs for 'Only Girl (In the World)':
